<a href="https://colab.research.google.com/github/k-vinamr/rera_construction_pred/blob/main/Streamlined_5th_floor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import joblib

In [ ]:
def load_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)
    return data

In [ ]:
def preprocess_data(data):
    # Convert date columns to datetime objects
    data['actual_commencement_date'] = pd.to_datetime(data['actual_commencement_date'], errors='coerce')
    data['estimated_finish_date'] = pd.to_datetime(data['estimated_finish_date'], errors='coerce')

    # Calculate derived columns
    current_date = datetime.now()
    data['duration_since_commencement'] = (current_date - data['actual_commencement_date']).dt.days
    data['remaining_duration'] = (data['estimated_finish_date'] - current_date).dt.days
    data['progress_ratio'] = data['duration_since_commencement'] / data['total_days']
    data['current_stage'] = data.apply(determine_status, axis=1)

    # Label encoding for 'current_stage' and 'project_state' columns
    label_encoder = LabelEncoder()
    data['current_stage'] = data['current_stage'].astype(str)
    data['current_stage_encoded'] = label_encoder.fit_transform(data['current_stage'])
    data['project_state'] = data['project_state'].astype(str)
    data['project_state_encoded'] = label_encoder.fit_transform(data['project_state'])

    return data

In [ ]:
# Define a function to determine the project status
def determine_status(row):
    current_date = datetime.now()
    if current_date > row['estimated_finish_date']:
        return 'completed'
    elif current_date >= row['actual_commencement_date']:
        return 'running'
    else:
        return 'upcoming'

# Apply the function to create the 'Status' column
    data['current_stage'] = data.apply(determine_status, axis=1)
    return data

In [ ]:
def feature_selection(data):
    # Select relevant features
    numerical_columns = data.select_dtypes(include=['int64', 'float64'])
    stage_columns = [
    'Cleaning & survey',
    'Excavation,leveling & P.C.C for Basement B1',
    'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring',
    'Slab of B (bottom)',
    'Ground Floor slab casting',
    '1st floor Columns casting',
    '1st Floor slab casting',
    '2nd floor Columns casting',
    '2nd Floor slab casting',
    'Brick work at Basement to Ground Floor',
    '3rd floor Columns casting',
    '3rd Floor slab casting',
    'Brick work at 1st Floor',
    'Electrical concealed, PVC Fitting, plastering at basement- ground floor',
    '4th floor Columns casting',
    '4th Floor slab casting',
    'Brick work at 2nd Floor',
    'Electrical concealed, PVC Fitting, plastering at 1st floor',
    '5th floor Columns casting',
    '5th Floor slab casting',
    'Brick work of 3rd to 5th Floor',
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor',
    'Plastering on outer sides',
    'Tiles work',
    'Painting and Finishing',
    'Plumbing & Sanitary,Electrification Works',
    'Doors & Windows Fixing Furniture work'
]

    # Target variables
    X = data[numerical_columns.columns.difference(stage_columns)].drop(columns='total_days')
    y_total_days = data['total_days']
    y_time_taken_at_each_stage = data[stage_columns]

    return X, y_total_days, y_time_taken_at_each_stage

In [ ]:
def train_evaluate_model(X_train, X_test, y_train, y_test):
    # Train and evaluate a Random Forest regression model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return model, mse, r2

In [ ]:
def determine_relevant_stage_and_columns(row, stage_columns):
    current_date = datetime.now()
    days_elapsed = row['days_elapsed']
    total_days = 0
    relevant_stage = None
    relevant_columns = []

    for stage in stage_columns:
        stage_duration = row[stage]  # Days required for the current stage
        total_days += stage_duration
        if total_days >= days_elapsed:
            relevant_stage = stage
            relevant_columns = [stage for stage in stage_columns[stage_columns.index(stage):]]
            break

    return relevant_stage, relevant_columns, total_days

In [ ]:
def make_predictions(row, stage_columns):
    if row['current_stage'] == 'completed':
        return None  # No predictions for completed projects
    elif row['current_stage'] == 'running':
        if row['relevant_stage'] is not None:
            if row['relevant_stage'] in row['relevant_columns']:
                return row[row['relevant_stage']] - (row['days_elapsed'] - row['total_days'])
            else:
                return None  # Skip stages before the relevant stage
        else:
            return None  # Skip predictions for running projects with no relevant stage
    elif row['current_stage'] == 'upcoming':
        return row['total_days']  # Include all stages for upcoming projects

## **Training Code**

In [ ]:
def main(file_path):
    data = load_data(file_path)
    data = preprocess_data(data)

    # Determine relevant stage and columns
    data['current_stage'] = data.apply(determine_status, axis=1)

    X, y_total_days, y_time_taken_at_each_stage = feature_selection(data)

    # Split the data for total days
    X_train, X_test, y_train_total_days, y_test_total_days = train_test_split(X, y_total_days, test_size=0.2, random_state=42)

    # Train and evaluate the model for total days
    total_days_model, mse_total_days, r2_total_days = train_evaluate_model(X_train, X_test, y_train_total_days, y_test_total_days)
    print(f"Mean Squared Error (Total Days): {mse_total_days}")
    print(f"R-squared (Total Days): {r2_total_days}")

    stage_columns = [
    'Cleaning & survey',
    'Excavation,leveling & P.C.C for Basement B1',
    'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring',
    'Slab of B (bottom)',
    'Ground Floor slab casting',
    '1st floor Columns casting',
    '1st Floor slab casting',
    '2nd floor Columns casting',
    '2nd Floor slab casting',
    'Brick work at Basement to Ground Floor',
    '3rd floor Columns casting',
    '3rd Floor slab casting',
    'Brick work at 1st Floor',
    'Electrical concealed, PVC Fitting, plastering at basement- ground floor',
    '4th floor Columns casting',
    '4th Floor slab casting',
    'Brick work at 2nd Floor',
    'Electrical concealed, PVC Fitting, plastering at 1st floor',
    '5th floor Columns casting',
    '5th Floor slab casting',
    'Brick work of 3rd to 5th Floor',
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor',
    'Plastering on outer sides',
    'Tiles work',
    'Painting and Finishing',
    'Plumbing & Sanitary,Electrification Works',
    'Doors & Windows Fixing Furniture work'
]

    # Split the data into training and testing sets
    X_train, X_test, y_train_total_days, y_test_total_days, y_train_time, y_test_time = train_test_split(X, y_total_days, y_time_taken_at_each_stage, test_size=0.2, random_state=42)

    # Create and train the regression model for time taken at each stage
    stage_models = {}
    for stage in stage_columns:
        stage_model = RandomForestRegressor(n_estimators=100, random_state=42)
        stage_model.fit(X_train, y_train_time[stage])
        stage_models[stage] = stage_model

    # Predict time taken at each stage on the test set
    y_pred_time = {}
    for stage, stage_model in stage_models.items():
        y_pred_time[stage] = stage_model.predict(X_test)
    current_date = datetime.now()
    data['days_elapsed'] = (current_date - data['actual_commencement_date']).dt.days

    # Determine relevant stage and columns
    data[['relevant_stage', 'relevant_columns', 'total_days']] = data.apply(lambda row: determine_relevant_stage_and_columns(row, stage_columns), axis=1, result_type='expand')

    data['relevant_stage'].fillna('Doors & Windows Fixing Furniture work', inplace=True)

    # Filter projects based on current_stage
    completed_projects = data[data['current_stage'] == 'completed']
    running_projects = data[data['current_stage'] == 'running']
    upcoming_projects = data[data['current_stage'] == 'upcoming']
    # Predict time taken for each project
    data['predicted_time_taken'] = data.apply(lambda row: make_predictions(row, stage_columns), axis=1)
    data = data.dropna()

    # Evaluate the models for time taken at each stage
    mse_stage = {}
    r2_stage = {}
    for stage in stage_columns:
        mse_stage[stage] = mean_squared_error(y_test_time[stage], y_pred_time[stage])
        r2_stage[stage] = r2_score(y_test_time[stage], y_pred_time[stage])
        print(f'Mean Squared Error ({stage}): {mse_stage[stage]}')
        print(f'R-squared ({stage}): {r2_stage[stage]}')

    # Save the total days model
    joblib.dump(total_days_model, 'total_days_model.joblib')

    # Save each stage model
    for stage, stage_model in stage_models.items():
        joblib.dump(stage_model, f'{stage}_model.joblib')

if __name__ == "__main__":
    file_path = 'augmented_5_floor.csv'
    main(file_path)

KeyError: ignored

In [ ]:
# Load the new data (replace 'new_data.csv' with the actual file path)
new_data = pd.read_csv('new_5_test_file.csv')


stage_columns = [
    'Cleaning & survey',
    'Excavation,leveling & P.C.C for Basement B1',
    'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring',
    'Slab of B (bottom)',
    'Ground Floor slab casting',
    '1st floor Columns casting',
    '1st Floor slab casting',
    '2nd floor Columns casting',
    '2nd Floor slab casting',
    'Brick work at Basement to Ground Floor',
    '3rd floor Columns casting',
    '3rd Floor slab casting',
    'Brick work at 1st Floor',
    'Electrical concealed, PVC Fitting, plastering at basement- ground floor',
    '4th floor Columns casting',
    '4th Floor slab casting',
    'Brick work at 2nd Floor',
    'Electrical concealed, PVC Fitting, plastering at 1st floor',
    '5th floor Columns casting',
    '5th Floor slab casting',
    'Brick work of 3rd to 5th Floor',
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor',
    'Plastering on outer sides',
    'Tiles work',
    'Painting and Finishing',
    'Plumbing & Sanitary,Electrification Works',
    'Doors & Windows Fixing Furniture work'
]

# Load the total days model
total_days_model = joblib.load('total_days_model.joblib')

# Load each stage model
stage_models = {}
for stage in stage_columns:
    stage_model = joblib.load(f'{stage}_model.joblib')
    stage_models[stage] = stage_model


# Converting date columns to datetime objeacts
new_data['actual_commencement_date'] = pd.to_datetime(new_data['actual_commencement_date'], errors='coerce')
new_data['estimated_finish_date'] = pd.to_datetime(new_data['estimated_finish_date'], errors='coerce')

current_date = datetime.now()

new_data['total_days'] = (new_data['estimated_finish_date'] - new_data['actual_commencement_date']).dt.days

new_data['duration_since_commencement'] = (current_date - new_data['actual_commencement_date']).dt.days
new_data['remaining_duration'] = (new_data['estimated_finish_date'] - current_date).dt.days
new_data['progress_ratio'] = new_data['duration_since_commencement'] / new_data['total_days']

# Define a function to determine the project status
def determine_status(row):
    if current_date > row['estimated_finish_date']:
        return 'completed'
    elif current_date >= row['actual_commencement_date']:
        return 'running'
    else:
        return 'upcoming'

# Apply the function to create the 'Status' column
new_data['current_stage'] = new_data.apply(determine_status, axis=1)

# Performing label encoding on the 'current_stage' column
label_encoder = LabelEncoder()
new_data['current_stage'] =new_data['current_stage'].astype(str) # Converting to string to handle any NaN values left
new_data['current_stage_encoded'] = label_encoder.fit_transform(new_data['current_stage'])

# Performing label encoding on the 'current_stage' column
label_encoder = LabelEncoder()
new_data['project_state'] =new_data['project_state'].astype(str) # Converting to string to handle any NaN values left
new_data['project_state_encoded'] = label_encoder.fit_transform(new_data['project_state'])

new_data['office_no'] = new_data['office_no'].astype(str)

# Create an empty DataFrame to store 'new_data' with added columns
new_data = new_data.copy()  # Assuming 'new_data' is your original DataFrame

# Add stage-related columns and 'total_days' column with NaN values
for stage in stage_columns:
    new_data[stage] = np.nan

new_data['total_days'] = np.nan

# Now, 'new_data_with_stages' contains the additional columns with NaN values


numerical_columns = new_data.select_dtypes(include=['int64', 'float64'])
stage_columns = [
    'Cleaning & survey',
    'Excavation,leveling & P.C.C for Basement B1',
    'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring',
    'Slab of B (bottom)',
    'Ground Floor slab casting',
    '1st floor Columns casting',
    '1st Floor slab casting',
    '2nd floor Columns casting',
    '2nd Floor slab casting',
    'Brick work at Basement to Ground Floor',
    '3rd floor Columns casting',
    '3rd Floor slab casting',
    'Brick work at 1st Floor',
    'Electrical concealed, PVC Fitting, plastering at basement- ground floor',
    '4th floor Columns casting',
    '4th Floor slab casting',
    'Brick work at 2nd Floor',
    'Electrical concealed, PVC Fitting, plastering at 1st floor',
    '5th floor Columns casting',
    '5th Floor slab casting',
    'Brick work of 3rd to 5th Floor',
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor',
    'Plastering on outer sides',
    'Tiles work',
    'Painting and Finishing',
    'Plumbing & Sanitary,Electrification Works',
    'Doors & Windows Fixing Furniture work'
]

# Target variables
X_new = new_data[numerical_columns.columns.difference(stage_columns)].drop(columns='total_days')
#y_total_days = new_data['total_days']
#y_time_taken_at_each_stage = new_data[stage_columns]  # Replace with your actual stage columns

new_data['total_days'] = total_days_model.predict(new_data[X_new.columns])

for stage in stage_columns:
    stage_model = stage_models.get(stage)  # Use get to avoid KeyError
    if stage_model:
        new_data[stage] = stage_model.predict(new_data[X_new.columns])
    else:
        # If the stage column isn't in stage_models, fill it with NaN
        new_data[stage] = np.nan

stage_materials = {
    'Cleaning & survey': [],
    'Excavation,leveling & P.C.C for Basement B1': ['cement', 'sand', 'aggregates'],
    'Raft footing, Column B1, Retaining wall Reinforcement ,Concrete pouring': ['TMT bar', 'cement', 'sand', 'aggregates'],
    'Slab of B (bottom)': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Ground Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '1st floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '1st Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work at Basement to Ground Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at basement- ground floor': ['circuit pipe', 'Cpvc', '&Pvc pipe', 'cement', 'sand'],
    '2nd floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '2nd Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work at 1st Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at 1st floor': ['circuit pipe', 'Cpvc', '&Pvc pipe', 'cement', 'sand'],
    '3rd floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '3rd Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work at 2nd Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor': ['cement', 'sand', 'circuit pipe', 'Cpvc', '&Pvc pipe'],
    '4th floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '4th Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work at 3rd to 5th Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor': ['cement', 'sand', 'circuit pipe', 'Cpvc', '&Pvc pipe'],
    '5th floor Columns casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    '5th Floor slab casting': ['TMT bar', 'cement', 'sand', 'aggregates', 'Pvc pipes', 'circuit pipes', 'lightbox', 'fan box'],
    'Brick work of 3rd to 5th Floor': ['blocks/bricks', 'cement', 'sand'],
    'Electrical concealed, PVC Fitting, plastering at 2nd-5th floor': ['cement', 'sand', 'circuit pipe', 'Cpvc', '&Pvc pipe'],
    'Plastering on outer sides': ['cement', 'sand'],
    'Tiles work': ['Tiles'],
    'Painting and Finishing': ['paint', 'putty', 'primer'],
    'Plumbing & Sanitary,Electrification Works': ['Wire', 'switch', 'nozzle Bib cock', 'shower', 'kitchen accessories', 'lights', 'fans', 'sheets', 'basin', 'sink', 'etc.'],
    'Doors & Windows Fixing Furniture work': ['Furniture', 'hardware', 'glass']
}


# Initialize empty lists to store relevant_stage and relevant_columns
relevant_stage_list = []
relevant_columns_list = []

# Iterate through each row (project) in new_data
for index, row in new_data.iterrows():
    days_elapsed = (current_date - row['actual_commencement_date']).days
    total_days = row['total_days']

    relevant_stage = None
    relevant_columns = []

    for stage in stage_columns:

        if total_days is None or np.isnan(total_days):
            # Set relevant_stage and relevant_columns to None if total_days is missing
            relevant_stage = None
            relevant_columns = []
            break

        stage_duration = row[stage]
        total_days -= stage_duration

        if total_days <= days_elapsed:
            relevant_stage = stage
            relevant_columns = [stage for stage in stage_columns[stage_columns.index(stage):]]
            break

    relevant_stage_list.append(relevant_stage)
    relevant_columns_list.append(relevant_columns)

    # Add the 'relevant_stage' and 'relevant_columns' columns to the new_data DataFrame
new_data['relevant_stage'] = relevant_stage_list
new_data['relevant_columns'] = relevant_columns_list


# Initialize empty DataFrame to store adjusted days data
adjusted_days_data = pd.DataFrame(columns=stage_columns)

# Iterate through each row (project) in new_data
for index, row in new_data.iterrows():
    current_stage = row['current_stage']
    relevant_columns = row['relevant_columns']

    if current_stage == 'completed':
        # For completed projects, set 'days' to NaN for all stage columns
        days_values = [np.nan] * len(stage_columns)
    elif current_stage == 'running':
        # For running projects, update 'days' based on relevant_columns
        days_values = [row[stage] if stage in relevant_columns else np.nan for stage in stage_columns]
    elif current_stage == 'upcoming':
        # For upcoming projects, keep the predicted 'days' values
        days_values = [row[stage] for stage in stage_columns]

    # Append the days_values to the adjusted_days_data
    adjusted_days_data = adjusted_days_data.append(pd.Series(days_values, index=stage_columns), ignore_index=True)


# Replace the original stage values in new_data with adjusted values
new_data[stage_columns] = adjusted_days_data

# Create a list to store the data
data = []

# Iterate through each row (project) in the new_data DataFrame
for index, row in new_data.iterrows():
    current_materials = []  # List to store materials for the current stage
    current_stage_start_date = current_date  # Initialize with the current date
    project_id = f'Project_{index + 1}'  # Generate a project_id

    for stage in stage_columns:
        stage_duration = row[stage]

        if not pd.isna(stage_duration):
            # Calculate the stage end date
            stage_end_date = current_stage_start_date + pd.Timedelta(days=stage_duration)

            # Format the dates for display
            start_date_str = current_stage_start_date.strftime('%m/%d/%Y')
            end_date_str = stage_end_date.strftime('%m/%d/%Y')

            # Check if there are materials for this stage
            materials = stage_materials.get(stage, [])

            if materials:
                # Store the data in the desired format
                data.append([end_date_str, stage, ", ".join(materials), project_id, row['organisation_name'], row['office_no'], row['project_res_no']])

            # Update the start date for the next stage
            current_stage_start_date = stage_end_date

# Create a DataFrame from the data
output_data = pd.DataFrame(data, columns=['Date', 'Stage', 'Recommended Materials', 'Project_ID', 'organisation_name', 'office_no', 'project_res_no'])

# Save the DataFrame to a CSV file
output_data.to_csv('output_5_final.csv', index=False)
